## Content Based Filtering


## For content based recommendation system the approach which I followed is described in short as follows:-

## 1. From movies.csv file i made another dataset which contains the movieId and the corresponding TFIDF vector with number of components limited to 23 (In my working enviroment in jupyter notebook the size of vector was limitedto 23).

## 2. Then using the file tags.csv I made a datframe which has userId as its index and all the genres watched by him grouped.

## 3. Using that datframe i made similar TFIDF vector dataframe of each user .

## 4. Then I calculated the cosine similarity of the user with all the movies vector and recommended the top 10 movies with the highest similarity scores.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import TruncatedSVD



In [2]:
!pip install gdown

In [3]:
import pandas as pd
import gdown

# Replace 'your_file_id' with the actual file ID of your CSV file
file_id = '1qRHxpEk2l7AgzDuORP33H1ZsPZFMeNBe'

# Construct the download URL
url = f'https://drive.google.com/uc?id={file_id}'

# Specify the output file name
output = 'movies.csv'

# Download the CSV file from Google Drive
gdown.download(url, output, quiet=False)

# Read the CSV file into a pandas DataFrame
df = pd.read_csv(output)

# Now you can work with the DataFrame 'df'
print(df.head())


Downloading...
From: https://drive.google.com/uc?id=1qRHxpEk2l7AgzDuORP33H1ZsPZFMeNBe
To: C:\Users\hp\movies.csv
100%|██████████| 3.04M/3.04M [00:00<00:00, 5.07MB/s]

   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy  


In [4]:
df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62423 entries, 0 to 62422
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  62423 non-null  int64 
 1   title    62423 non-null  object
 2   genres   62423 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.4+ MB


In [6]:
df['genres']=df['genres'].str.replace('|', ' ')
df.head()

C:\Users\hp\AppData\Local\Temp\ipykernel_98740\4194037766.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['genres']=df['genres'].str.replace('|', ' ')


,movieId,title,genres
0,1,Toy Story (1995),Adventure Animation Children Comedy Fantasy
1,2,Jumanji (1995),Adventure Children Fantasy
2,3,Grumpier Old Men (1995),Comedy Romance
3,4,Waiting to Exhale (1995),Comedy Drama Romance
4,5,Father of the Bride Part II (1995),Comedy


In [7]:
# Create TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words='english')

# Fit and transform TF-IDF vectorizer
tfidf_matrix = tfidf_vectorizer.fit_transform(df['genres'])
# Apply TruncatedSVD to reduce dimensionality
svd = TruncatedSVD(n_components=23)
tfidf_matrix_svd = svd.fit_transform(tfidf_matrix)

# Compute cosine similarity on reduced matrix
#cosine_sim = cosine_similarity(tfidf_matrix_svd, tfidf_matrix_svd)




In [8]:
movie_ids = df['movieId']

In [9]:
movie_tfidf_df = pd.DataFrame({
    'movieId': movie_ids,
    'TFIDF_Vector': tfidf_matrix_svd.tolist()  # Convert TF-IDF matrix rows to lists
    
})

movie_tfidf_df.head()


,movieId,TFIDF_Vector
0,1,"[0.15608624759187623, 0.29825098107598697, 0.0..."
1,2,"[0.050221738947306395, 0.0429121320115672, 0.0..."
2,3,"[0.3901076181648996, 0.6325595403256057, -0.02..."
3,4,"[0.7385266991807968, 0.3830496338222682, -0.03..."
4,5,"[0.38610991762419505, 0.8962018299711101, -0.0..."


In [10]:
movie_tfidf_df['TFIDF_Vector'].dtype

dtype('O')

In [11]:
tfidf_matrix_svd


array([[ 1.56086248e-01,  2.98250981e-01,  7.42351253e-03, ...,
        -1.81296903e-16, -7.99346746e-17, -3.23855633e-17],
       [ 5.02217389e-02,  4.29121320e-02,  1.16586522e-02, ...,
        -1.39456700e-16, -1.46412646e-16, -4.40977966e-17],
       [ 3.90107618e-01,  6.32559540e-01, -2.62065974e-02, ...,
         1.83348964e-17, -1.31365088e-17,  3.01842000e-17],
       ...,
       [ 8.57593973e-01,  4.27060732e-01, -2.51096114e-02, ...,
        -7.87263794e-18,  2.22759001e-17, -9.56273371e-18],
       [ 1.30725914e-14, -3.72325615e-14,  5.18110867e-12, ...,
        -1.66533454e-16,  0.00000000e+00, -1.11022302e-16],
       [ 3.93745799e-01, -1.26306707e-01,  1.92131322e-02, ...,
        -1.09357008e-16,  6.29757964e-19,  6.02500541e-17]])

In [12]:
tags_df=pd.read_csv('tags.csv')

In [13]:
# Replace 'your_file_id' with the actual file ID of your CSV file
file_id = '1YcoWZBKSIXRA34hKsSkk25jFt86DMx-4'

# Construct the download URL
url = f'https://drive.google.com/uc?id={file_id}'

# Specify the output file name
output = 'tags.csv'

# Download the CSV file from Google Drive
gdown.download(url, output, quiet=False)

# Read the CSV file into a pandas DataFrame
tags_df = pd.read_csv(output)

# Now you can work with the DataFrame 'df'
tags_df.head()

Downloading...
From: https://drive.google.com/uc?id=1YcoWZBKSIXRA34hKsSkk25jFt86DMx-4
To: C:\Users\hp\tags.csv
100%|██████████| 38.8M/38.8M [00:07<00:00, 5.17MB/s]


,userId,movieId,tag,timestamp
0,3,260,classic,1439472355
1,3,260,sci-fi,1439472256
2,4,1732,dark comedy,1573943598
3,4,1732,great dialogue,1573943604
4,4,7569,so bad it's good,1573943455


In [14]:
tags_df.head()

,userId,movieId,tag,timestamp
0,3,260,classic,1439472355
1,3,260,sci-fi,1439472256
2,4,1732,dark comedy,1573943598
3,4,1732,great dialogue,1573943604
4,4,7569,so bad it's good,1573943455


In [15]:

# Group historical preference data by userId and aggregate movie tags

user_profiles = tags_df.groupby('userId')['tag'].apply(lambda x: ' '.join(str(i) for i in x)).reset_index()


In [16]:
user_profiles.head()

,userId,tag
0,3,classic sci-fi
1,4,dark comedy great dialogue so bad it's good un...
2,19,Oscar (Best Supporting Actress) adventure anim...
3,20,bah
4,43,Clint Eastwood


In [17]:


# Fit and transform TF-IDF vectorizer
tfidf_preference_matrix = tfidf_vectorizer.fit_transform(user_profiles['tag'])
# Apply TruncatedSVD to reduce dimensionality
svd = TruncatedSVD(n_components=23)
tfidf_prefernce_matrix_svd = svd.fit_transform(tfidf_preference_matrix)



In [18]:
tfidf_prefernce_matrix_svd.shape 

(14592, 23)

In [19]:
tfidf_prefernce_matrix_svd


array([[ 5.64363887e-01,  6.07700103e-01, -4.05955450e-01, ...,
         2.83086067e-02, -3.66484203e-02, -4.35286604e-02],
       [ 1.74461307e-01, -1.61137380e-01, -7.87701925e-03, ...,
        -1.74426213e-02, -8.30715935e-02, -3.95637424e-03],
       [ 1.10642696e-01, -3.00797254e-02,  8.36691143e-02, ...,
        -1.73418934e-02, -5.70679933e-02, -2.91289552e-02],
       ...,
       [ 5.49348911e-02, -4.37700470e-02,  4.64848333e-03, ...,
         1.10363138e-01, -8.16818754e-03, -6.28895846e-02],
       [ 4.73725109e-03, -3.44519720e-03,  7.56098977e-04, ...,
        -1.44572374e-04, -1.62401033e-03, -1.27057838e-03],
       [ 5.88994349e-02, -6.94189475e-02, -3.95822669e-03, ...,
         3.60057393e-03, -1.45841743e-02, -7.32246595e-03]])

In [20]:


# Assuming tfidf_prefernce_matrix_svd contains the TF-IDF matrix for users
# and each row represents a user and each column represents a feature

# Extracting user IDs
user_ids = user_profiles['userId']

# Creating a DataFrame with user IDs and TF-IDF vectors
user_tfidf_df = pd.DataFrame({
    'userId': user_ids,
    'TFIDF_Vector': tfidf_prefernce_matrix_svd.tolist()  # Convert TF-IDF matrix rows to lists
})

# Display the DataFrame
user_tfidf_df.head()


,userId,TFIDF_Vector
0,3,"[0.5643638870982318, 0.6077001032498723, -0.40..."
1,4,"[0.1744613067641897, -0.16113738045187212, -0...."
2,19,"[0.11064269599288999, -0.03007972537832434, 0...."
3,20,"[-2.651221955200311e-16, -1.329253162344432e-1..."
4,43,"[0.012890064339920716, -0.011067666863988815, ..."


In [21]:
user_tfidf_df

,userId,TFIDF_Vector
0,3,"[0.5643638870982318, 0.6077001032498723, -0.40..."
1,4,"[0.1744613067641897, -0.16113738045187212, -0...."
2,19,"[0.11064269599288999, -0.03007972537832434, 0...."
3,20,"[-2.651221955200311e-16, -1.329253162344432e-1..."
4,43,"[0.012890064339920716, -0.011067666863988815, ..."
...,...,...
14587,162495,"[0.05592647847557776, -0.06785425299744749, -0..."
14588,162501,"[0.015848874745174695, -0.016228750543782172, ..."
14589,162512,"[0.054934891135461864, -0.043770046989130916, ..."
14590,162521,"[0.00473725108697616, -0.0034451971956205906, ..."


In [24]:

from sklearn.metrics.pairwise import cosine_similarity
def parse_array(array_str):
    # Remove brackets and split by comma, then convert to float
    return np.array(array_str.strip('[]').split(', ')).astype(float)
def parse_array(array):
    return np.array(array).astype(float)

# Applying the function to convert string arrays to numeric arrays
movie_tfidf_df['TFIDF_Vector'] = movie_tfidf_df['TFIDF_Vector'].apply(parse_array)


user_id = 3
user_vector = user_tfidf_df.loc[user_tfidf_df['userId'] == user_id, 'TFIDF_Vector'].iloc[0]

# Calculating cosine similarity with each movie
similarities = {}
for index, row in movie_tfidf_df.iterrows():
    movie_id = row['movieId']
    movie_vector = row['TFIDF_Vector']
    similarity = cosine_similarity([user_vector], [movie_vector])[0][0]
    similarities[movie_id] = similarity

# Printing the similarities
for movie_id, similarity in similarities.items():
    print(f"Cosine Similarity between user {user_id} and movie {movie_id}: {similarity}")




IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Cosine Similarity between user 3 and movie 199107: 0.21717406999018257
Cosine Similarity between user 3 and movie 199109: 0.20688724292929078
Cosine Similarity between user 3 and movie 199111: 0.3156938415529804
Cosine Similarity between user 3 and movie 199113: 0.7459482257417067
Cosine Similarity between user 3 and movie 199115: 0.23752413383184492
Cosine Similarity between user 3 and movie 199117: 0.23752413383184492
Cosine Similarity between user 3 and movie 199119: 0.17740412251245508
Cosine Similarity between user 3 and movie 199121: 0.3156938415529804
Cosine Similarity between user 3 and movie 199123: 0.7081438760800516
Cosine Similarity between user 3 and movie 199125: 0.20688724292929078
Cosine Similarity between user 3 and movie 199127: 0.23752413383184492
Cosine Similarity between user 3 and movie 199129: 0.23752413383184492
Cosine Similarity between user 3 and movie 199131: 0.0632074821705112
Cosine Similarity between user 3 and movie 199133: 0.23752413383184492
Cosine Simi

In [26]:
# Sort the similarities dictionary by values (similarity scores)
sorted_similarities = sorted(similarities.items(), key=lambda x: x[1], reverse=True)

# Creating a dictionary to map movie IDs to movie names
movie_id_to_name = dict(zip(df['movieId'], df['title']))

# Print the names of the top 10 movies with highest similarity scores
print("Top 10 movies with highest similarity scores:")
for i, (movie_id, similarity) in enumerate(sorted_similarities[:10], 1):
    movie_name = movie_id_to_name.get(movie_id, "Unknown")
    print(f"{i}. Movie ID: {movie_id}, Movie Name: {movie_name}")


Top 10 movies with highest similarity scores:
1. Movie ID: 5, Movie Name: Father of the Bride Part II (1995)
2. Movie ID: 18, Movie Name: Four Rooms (1995)
3. Movie ID: 19, Movie Name: Ace Ventura: When Nature Calls (1995)
4. Movie ID: 65, Movie Name: Bio-Dome (1996)
5. Movie ID: 69, Movie Name: Friday (1995)
6. Movie ID: 88, Movie Name: Black Sheep (1996)
7. Movie ID: 102, Movie Name: Mr. Wrong (1996)
8. Movie ID: 104, Movie Name: Happy Gilmore (1996)
9. Movie ID: 115, Movie Name: Happiness Is in the Field (Bonheur est dans le pré, Le) (1995)
10. Movie ID: 119, Movie Name: Steal Big, Steal Little (1995)


## Collaborative filtering


In [27]:
!pip install scikit-surprise

In [28]:
conda install -c conda-forge scikit-surprise


Retrieving notices: ...working... done
Solving environment: ...working... done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.




==> WARNING: A newer version of conda exists. <==
  current version: 23.1.0
  latest version: 24.4.0

Please update conda by running

    $ conda update -n base -c defaults conda

Or to minimize the number of packages updated during conda update use

     conda install conda=24.4.0




In [29]:
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split

In [30]:
# Replace 'your_file_id' with the actual file ID of your CSV file
file_id = '1xSNqvnmt61DhSB2c2ly1DFXM9tyIlLL0'

# Construct the download URL
url = f'https://drive.google.com/uc?id={file_id}'

# Specify the output file name
output = 'ratings.csv'

# Download the CSV file from Google Drive
gdown.download(url, output, quiet=False)

# Read the CSV file into a pandas DataFrame
ratings_df = pd.read_csv(output)

# Now you can work with the DataFrame 'df'
ratings_df.head()

Downloading...
From (original): https://drive.google.com/uc?id=1xSNqvnmt61DhSB2c2ly1DFXM9tyIlLL0
From (redirected): https://drive.google.com/uc?id=1xSNqvnmt61DhSB2c2ly1DFXM9tyIlLL0&confirm=t&uuid=3e719b75-4f6c-4eb5-94f9-02351feb603e
To: C:\Users\hp\ratings.csv
100%|██████████| 678M/678M [01:55<00:00, 5.86MB/s] 


,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510


In [31]:
from scipy.sparse import coo_matrix
user_ids = ratings_df['userId'].unique()
movie_ids = ratings_df['movieId'].unique()

# Create a mapping from user IDs and movie IDs to matrix indices
user_id_map = {user_id: i for i, user_id in enumerate(user_ids)}
movie_id_map = {movie_id: i for i, movie_id in enumerate(movie_ids)}

# Convert user IDs and movie IDs to matrix indices
row = ratings_df['userId'].map(user_id_map)
col = ratings_df['movieId'].map(movie_id_map)
data = ratings_df['rating']

# Create a COO matrix
user_item_matrix = coo_matrix((data, (row, col)), shape=(len(user_ids), len(movie_ids)))

print(user_item_matrix)

  (0, 0)	5.0
  (0, 1)	3.5
  (0, 2)	5.0
  (0, 3)	5.0
  (0, 4)	3.5
  (0, 5)	4.0
  (0, 6)	3.5
  (0, 7)	3.5
  (0, 8)	5.0
  (0, 9)	4.0
  (0, 10)	3.5
  (0, 11)	4.0
  (0, 12)	2.5
  (0, 13)	2.5
  (0, 14)	2.5
  (0, 15)	3.5
  (0, 16)	4.5
  (0, 17)	4.0
  (0, 18)	5.0
  (0, 19)	5.0
  (0, 20)	4.5
  (0, 21)	4.0
  (0, 22)	5.0
  (0, 23)	5.0
  (0, 24)	5.0
  :	:
  (162540, 422)	4.5
  (162540, 429)	3.0
  (162540, 3087)	4.5
  (162540, 4800)	0.5
  (162540, 440)	2.0
  (162540, 6218)	3.5
  (162540, 225)	4.5
  (162540, 442)	4.5
  (162540, 227)	5.0
  (162540, 2369)	1.5
  (162540, 11215)	5.0
  (162540, 48)	4.5
  (162540, 3839)	2.5
  (162540, 12783)	3.0
  (162540, 236)	4.0
  (162540, 473)	4.5
  (162540, 1851)	4.5
  (162540, 245)	4.0
  (162540, 511)	4.0
  (162540, 521)	4.5
  (162540, 544)	4.5
  (162540, 4574)	2.5
  (162540, 7115)	2.0
  (162540, 578)	4.0
  (162540, 604)	5.0


In [73]:
import pandas as pd
from scipy.sparse import coo_matrix

# Your code to create the COO matrix
user_ids = ratings_df['userId'].unique()
movie_ids = ratings_df['movieId'].unique()

# Create a mapping from user IDs and movie IDs to matrix indices
user_id_map = {user_id: i for i, user_id in enumerate(user_ids)}
movie_id_map = {movie_id: i for i, movie_id in enumerate(movie_ids)}

# Convert user IDs and movie IDs to matrix indices
row = ratings_df['userId'].map(user_id_map)
col = ratings_df['movieId'].map(movie_id_map)
data = ratings_df['rating']

# Create a COO matrix
user_item_matrix = coo_matrix((data, (row, col)), shape=(len(user_ids), len(movie_ids)))

# Convert the COO matrix to a DataFrame
user_item_df = pd.DataFrame.sparse.from_spmatrix(user_item_matrix, index=user_ids, columns=movie_ids)

# Set row and column names
user_item_df.index.name = 'userid'
user_item_df.columns.name = 'movieid'

# Display the DataFrame
user_item_df



movieid,296,306,307,665,899,1088,1175,1217,1237,1250,...,200178,200182,200186,200188,200190,200192,200194,139970,200726,200728
userid,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.5,5.0,5.0,3.5,4.0,3.5,3.5,5.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,5.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162537,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
162538,2.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
162539,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [51]:
import pandas as pd

# Assuming you have your DataFrame user_item_df containing the ratings data

# Split the DataFrame into four parts
quarter_index_1 = len(user_item_df) // 4
quarter_index_2 = len(user_item_df) // 2
quarter_index_3 = 3 * len(user_item_df) // 4

first_quarter = user_item_df.iloc[:quarter_index_1]
second_quarter = user_item_df.iloc[quarter_index_1:quarter_index_2]
third_quarter = user_item_df.iloc[quarter_index_2:quarter_index_3]
fourth_quarter = user_item_df.iloc[quarter_index_3:]

# Now you have four quarters of the DataFrame
print("First quarter of the DataFrame:")
first_quarter



First quarter of the DataFrame:


movieid,296,306,307,665,899,1088,1175,1217,1237,1250,...,200178,200182,200186,200188,200190,200192,200194,139970,200726,200728
userid,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.5,5.0,5.0,3.5,4.0,3.5,3.5,5.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,5.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40631,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
40632,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
40633,2.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [52]:
print("\nSecond quarter of the DataFrame:")
second_quarter



Second quarter of the DataFrame:


movieid,296,306,307,665,899,1088,1175,1217,1237,1250,...,200178,200182,200186,200188,200190,200192,200194,139970,200726,200728
userid,,,,,,,,,,,,,,,,,,,,,
40636,4.0,3.0,3.0,0.0,0.0,2.0,3.0,0.0,2.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
40637,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
40638,3.5,3.5,3.5,0.0,3.0,2.0,3.5,0.0,3.5,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
40639,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
40640,5.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81266,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
81267,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
81268,4.5,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [53]:
print("\nThird quarter of the DataFrame:")
third_quarter



Third quarter of the DataFrame:


movieid,296,306,307,665,899,1088,1175,1217,1237,1250,...,200178,200182,200186,200188,200190,200192,200194,139970,200726,200728
userid,,,,,,,,,,,,,,,,,,,,,
81271,4.5,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
81272,3.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
81273,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
81274,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
81275,4.0,0.0,0.0,0.0,5.0,3.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121901,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
121902,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
121903,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [54]:
print("\nFourth quarter of the DataFrame:")
fourth_quarter


Fourth quarter of the DataFrame:


movieid,296,306,307,665,899,1088,1175,1217,1237,1250,...,200178,200182,200186,200188,200190,200192,200194,139970,200726,200728
userid,,,,,,,,,,,,,,,,,,,,,
121906,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
121907,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
121908,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
121909,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
121910,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162537,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
162538,2.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
162539,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [74]:
import pandas as pd

# Assuming you have your DataFrame user_item_df containing the ratings data

# Calculate the indices to split the DataFrame into 8 parts
split_indices = [len(user_item_df) * i // 8 for i in range(1, 8)]

# Split the DataFrame into eight parts
data_splits = []
start_index = 0
for end_index in split_indices:
    data_splits.append(user_item_df.iloc[start_index:end_index])
    start_index = end_index

# Add the last part
data_splits.append(user_item_df.iloc[start_index:])

# Now you have eight parts of the DataFrame
for i, split in enumerate(data_splits):
    print(f"Part {i+1} of the DataFrame:")
    print(split)


Part 1 of the DataFrame:
movieid  296     306     307     665     899     1088    1175    1217    \
userid                                                                    
1           5.0     3.5     5.0     5.0     3.5     4.0     3.5     3.5   
2           0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
3           5.0     0.0     0.0     0.0     0.0     0.0     0.0     5.0   
4           4.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
5           4.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
...         ...     ...     ...     ...     ...     ...     ...     ...   
20313       0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
20314       0.5     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
20315       3.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
20316       5.0     0.0     0.0     0.0     4.0     0.0     0.0     4.0   
20317       4.5     0.0     0.0     0.0     0.0     0.0     0.0     0.0   


In [77]:
import pandas as pd
from surprise import Dataset, Reader, KNNBasic

# Assuming you have your DataFrame user_item_df containing the ratings data

# Convert the pivot table back to long format
user_item_df_long = user_item_df.stack().reset_index()
user_item_df_long.columns = ['userID', 'movieID', 'rating']

# Calculate the indices to split the DataFrame into 8 parts
split_indices = [len(user_item_df_long) * i // 8 for i in range(1, 8)]

# Split the DataFrame into eight parts
data_splits = []
start_index = 0
for end_index in split_indices:
    data_splits.append(user_item_df_long.iloc[start_index:end_index])
    start_index = end_index

# Add the last part
data_splits.append(user_item_df_long.iloc[start_index:])

# Define a function to generate recommendations for a given segment
def generate_recommendations(segment_df, user_id):
    # Load the dataset from the DataFrame segment
    reader = Reader(rating_scale=(0, 5))  # Assuming ratings range from 0 to 5
    data = Dataset.load_from_df(segment_df[['userID', 'movieID', 'rating']], reader)
    
    # Use user-based collaborative filtering with KNNBasic algorithm
    sim_options = {'name': 'cosine', 'user_based': True}  # Use cosine similarity between users
    knn_model = KNNBasic(sim_options=sim_options)
    
    # Train the model
    trainset = data.build_full_trainset()
    knn_model.fit(trainset)
    
    # Find the most similar user within the segment
    user_similarity_matrix = knn_model.compute_similarities()
    most_similar_user_id = user_similarity_matrix[user_id].argmax()
    
    # Get the ratings of the most similar user within the segment
    most_similar_user_ratings = segment_df[segment_df['userID'] == most_similar_user_id]
    
    # Find movies that the most similar user rated highly and the target user hasn't rated yet
    movies_to_recommend = most_similar_user_ratings[most_similar_user_ratings['rating'] > 4]['movieID'].values
    target_user_rated_movies = segment_df[segment_df['userID'] == user_id]['movieID'].values
    movies_to_recommend = [movie_id for movie_id in movies_to_recommend if movie_id not in target_user_rated_movies]
    
    # Get top 10 recommended movie IDs
    top_10_recommendations = movies_to_recommend[:10]
    
    return top_10_recommendations

# Let's say we want to generate recommendations for user with userID = 121906
user_id = 121906

# Iterate through each segment and generate recommendations
for i, segment_df in enumerate(data_splits):
    recommendations = generate_recommendations(segment_df, user_id)
    print(f"Recommendations for user {user_id} in Segment {i+1}:")
    print(recommendations)


MemoryError: Unable to allocate 35.8 GiB for an array with shape (9597558427,) and data type int32

## As it can be seen above that even after divideing the datsets into subsets i was getting memory error due to large size of dataset. I was unable to do Collaborative filtering .

## The approach which i had though to implement was to make a matrix which has  user id as rows and movieid as column and then based upon the ratings i will find the most similar user  using K nearest neighbour. Then the recommendations will be generated.

## I was able to create the above matrix but unable to procedd further